In [72]:
import re
import copy
import os
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from langchain.schema import Document
from langchain_pinecone import PineconeVectorStore
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import Pinecone
from langchain_experimental.text_splitter import SemanticChunker

In [73]:
# process .env file
load_dotenv()

True

In [74]:
# Access the environment variables
openai_api_key = os.getenv('FINE_TUNED_TOXIC_DETECTION_API_KEY')
pinecone_api_key = os.getenv('PINECONE_API_KEY')
pc_index = os.getenv('PINECONE_GPT')
pc_semi_index = os.getenv('PINECONE_GPT_SEMI')

In [75]:
embeddings = OpenAIEmbeddings()
text_splitter = SemanticChunker(OpenAIEmbeddings())

In [91]:
documents = []
df = pd.read_csv('./data/csv/semi_toxic_classification_evaluation_set.csv')
df

,Text,Toxic
0,Date When did this algorithm get invented ?,0
1,answered them many times ! but they didnt answ...,0
2,IAC There is a discussion on article 's talk p...,0
3,Spandam 's laugh ? I 'm listening to it and al...,0
4,AND HAVING A FUCKIN ' BITCH ASS DAY '' ',1
...,...,...
995,"Oh , well Harry finally got his payback on me ...",1
996,this page is about random crap talk .. just ed...,1
997,I 've reverted the editsBOTH OF THEM VIOLATED ...,0
998,"I used the exact same bullshit you wrote , hyp...",1


In [92]:
# for index, row in df.iterrows():

#     # Example of creating a document with text and metadata
#     documents.append(Document(
#         page_content=f"{row['Text']} - Label: {row['Toxic']}"
#     ))

In [93]:
splits = text_splitter.create_documents([df['Text'][2]])
splits

[Document(metadata={}, page_content="IAC There is a discussion on article 's talk page .")]

In [94]:
i = 0
for index, row in df.iterrows():
    print(f'Index: {i}')
    splits = text_splitter.create_documents([row['Text']])
    formatted_splits = [
        Document(
            page_content=f"Text: {doc.page_content} - Label: {row['Toxic']}"
        ) for doc in splits
    ]
    documents.extend(formatted_splits)
    i += 1

Index: 0
Index: 1
Index: 2
Index: 3
Index: 4
Index: 5
Index: 6
Index: 7
Index: 8
Index: 9
Index: 10
Index: 11
Index: 12
Index: 13
Index: 14
Index: 15
Index: 16
Index: 17
Index: 18
Index: 19
Index: 20
Index: 21
Index: 22
Index: 23
Index: 24
Index: 25
Index: 26
Index: 27
Index: 28
Index: 29
Index: 30
Index: 31
Index: 32
Index: 33
Index: 34
Index: 35
Index: 36
Index: 37
Index: 38
Index: 39
Index: 40
Index: 41
Index: 42
Index: 43
Index: 44
Index: 45
Index: 46
Index: 47
Index: 48
Index: 49
Index: 50
Index: 51
Index: 52
Index: 53
Index: 54
Index: 55
Index: 56
Index: 57
Index: 58
Index: 59
Index: 60
Index: 61
Index: 62
Index: 63
Index: 64
Index: 65
Index: 66
Index: 67
Index: 68
Index: 69
Index: 70
Index: 71
Index: 72
Index: 73
Index: 74
Index: 75
Index: 76
Index: 77
Index: 78
Index: 79
Index: 80
Index: 81
Index: 82
Index: 83
Index: 84
Index: 85
Index: 86
Index: 87
Index: 88
Index: 89
Index: 90
Index: 91
Index: 92
Index: 93
Index: 94
Index: 95
Index: 96
Index: 97
Index: 98
Index: 99
Index: 100

In [95]:
len(documents)

1547

In [96]:
documents

[Document(metadata={}, page_content='Text: Date When did this algorithm get invented ? - Label: 0'),
 Document(metadata={}, page_content='Text: answered them many times ! but they didnt answer me , they just doing their vandalism and changing the correct - Label: 0'),
 Document(metadata={}, page_content="Text: IAC There is a discussion on article 's talk page . - Label: 0"),
 Document(metadata={}, page_content="Text: Spandam 's laugh ? I 'm listening to it and all I hear is da . - Label: 0"),
 Document(metadata={}, page_content='Text: Was it written as wa in the manga or something ? - Label: 0'),
 Document(metadata={}, page_content="Text: AND HAVING A FUCKIN ' BITCH ASS DAY '' ' - Label: 1"),
 Document(metadata={}, page_content='Text: I mean it . Please adjust this box to work with the rest of this page . And as for the time of these comments , ignore them . - Label: 0'),
 Document(metadata={}, page_content='Text: My computer clock is screwy . - Label: 0'),
 Document(metadata={}, page_

In [97]:
# insert splits into Pinecone vector database as embeddings
PineconeVectorStore.from_documents(documents, embeddings, index_name=pc_semi_index)